In [1]:
import numpy as np

In [2]:
gate1 = np.ndarray(shape=(2,2,2,2))
gate2 = np.ndarray(shape=(2,2,2,2))

In [3]:
gate2

array([[[[9.4593255e-317, 0.0000000e+000],
         [0.0000000e+000, 0.0000000e+000]],

        [[0.0000000e+000, 0.0000000e+000],
         [0.0000000e+000, 0.0000000e+000]]],


       [[[0.0000000e+000, 0.0000000e+000],
         [0.0000000e+000, 0.0000000e+000]],

        [[0.0000000e+000, 0.0000000e+000],
         [0.0000000e+000, 0.0000000e+000]]]])

In [4]:
def contraction4x4(gate1, gate2):
    shape1 = gate1.shape
    gate1_contraction = gate1.reshape(shape1[0]*shape1[1]*shape1[2], shape1[3])
    shape2 = gate2.shape
    gate2_contraction = gate2.reshape(shape2[0], shape2[1]*shape2[2]*shape2[3])
    contraction = np.dot(gate1_contraction, gate2_contraction)
    final_form = contraction.reshape(shape1[0], shape1[1], shape1[2], shape2[1], shape2[2], shape2[3])
    return final_form

In [5]:
c_4x4 = contraction4x4(gate1, gate2)
c_4x4.shape

(2, 2, 2, 2, 2, 2)

In [6]:
def contraction_vertical(gate1, gate2):
    shape1 = gate1.shape
    gate1_contraction = gate1.reshape(shape1[0]*shape1[1]*shape1[2], shape1[3])
    gate2_contraction = gate2.transpose(1,0,2,3)
    shape2 = gate2.shape
    gate2_contraction = gate2_contraction.reshape(shape2[0], shape2[1]*shape2[2]*shape2[3])
    contraction = np.dot(gate1_contraction, gate2_contraction)
    contraction = contraction.reshape(shape1[0], shape1[1], shape1[2], shape2[1], shape2[2], shape2[3])
    contraction = contraction.transpose(0,3,1,2,4,5)
    shapeC = contraction.shape
    final_form = contraction.reshape(shapeC[0]*shapeC[1], shapeC[2], shapeC[3]*shapeC[4], shapeC[5])
    return final_form

In [7]:
c_vertical = contraction_vertical(gate1, gate2)
c_vertical.shape

(4, 2, 4, 2)

In [8]:
def contraction_horizontal(gate1, gate2):
    gate1_contraction = gate1.transpose(0,1,3,2)
    shape1 = gate1_contraction.shape
    gate1_contraction = gate1_contraction.reshape(shape1[0]*shape1[1]*shape1[2], shape1[3])
    shape2 = gate2.shape
    gate2_contraction = gate2.reshape(shape2[0], shape2[1]*shape2[2]*shape2[3])
    contraction = np.dot(gate1_contraction, gate2_contraction)
    contraction = contraction.reshape(shape1[0], shape1[1], shape1[2], shape2[1], shape2[2], shape2[3])
    contraction = contraction.transpose(0,1,3,4,2,5)
    shapeC = contraction.shape
    final_form = contraction.reshape(shapeC[0], shapeC[1]*shapeC[2], shapeC[3], shapeC[4]*shapeC[5])
    return final_form

In [9]:
c_horizontal = contraction_horizontal(c_vertical, c_vertical)
c_horizontal.shape

(4, 4, 4, 4)

In [14]:
def run_test():
    c = gate1
    for i in range(3):
        c = contraction_vertical(c, c)
        print(c.shape)
        c = contraction_horizontal(c, c)
        print(c.shape)

In [15]:
import cProfile
import re
cProfile.run('run_test()')

(4, 2, 4, 2)
(4, 4, 4, 4)
(16, 4, 16, 4)
(16, 16, 16, 16)
(256, 16, 256, 16)
         205 function calls in 0.138 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        6    0.000    0.000    0.050    0.008 <__array_function__ internals>:2(dot)
        1    0.000    0.000    0.138    0.138 <ipython-input-14-76ac5a889385>:1(run_test)
        3    0.000    0.000    0.103    0.034 <ipython-input-6-ccdc64f7ae11>:1(contraction_vertical)
        3    0.000    0.000    0.034    0.011 <ipython-input-8-172de626b5e3>:1(contraction_horizontal)
        1    0.000    0.000    0.138    0.138 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _exceptions.py:146(__init__)
       11    0.000    0.000    0.001    0.000 iostream.py:197(schedule)
       10    0.000    0.000    0.000    0.000 iostream.py:309(_is_master_process)
       10    0.000    0.000    0.000    0.000 iostream.py:322(_schedule_flush)
       10    0.000    

MemoryError: Unable to allocate 32.0 GiB for an array with shape (65536, 65536) and data type float64

In [33]:
def truncation_horizontal(gate1, gate2, dim_tol):
    gate1_contraction = gate1.transpose(0,1,3,2)
    shape1 = gate1_contraction.shape
    gate1_contraction = gate1_contraction.reshape(shape1[0]*shape1[1]*shape1[2], shape1[3])
    shape2 = gate2.shape
    gate2_contraction = gate2.reshape(shape2[0], shape2[1]*shape2[2]*shape2[3])
    contraction = np.dot(gate1_contraction, gate2_contraction)
    
    [u,s,v] = np.linalg.svd(contraction)

    u = u[:,:dim_tol]
    v = v[:dim_tol,:]
    
    u_final = u.reshape(shape1[0], shape1[1], shape1[2], dim_tol)
    v_final = v.reshape(dim_tol, shape2[1], shape2[2], shape2[3])
    
    return u_final, v_final

In [36]:
u, v = truncation_horizontal(gate1, gate2, 2)
print(u.shape)
print(v.shape)

(2, 2, 2, 2)
(2, 2, 2, 2)


In [37]:
c = contraction_vertical(gate1, gate2)
print(c.shape)
u, v = truncation_horizontal(c, c, 2)
print(u.shape)
print(v.shape)

(4, 2, 4, 2)
(4, 2, 2, 2)
(2, 2, 4, 2)
